# Лабораторная работа №1
## Выполнил студент группы ГРУППА ФИО

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
3. [Задание 3](#Задание-№3)
4. [Вывод](#Вывод)

### Задание №1
<i> Вызвать функцию print() и передать туда строку Hello, World! </i>

### Задание №2
Написать генератор случайных матриц(многомерных), который принимает
опциональные параметры <b>m</b>, <b>n</b>, <b>min_limit</b>, <b>max_limit</b>, где <b>m</b> и <b>n</b> указывают размер
матрицы, а <b>min_lim</b> и <b>max_lim</b> - минимальное и максимальное значение для
генерируемого числа.

In [1]:
import time
from random import randint

In [3]:
user_m = int(input())
user_n = int(input())
user_min_limit = int(input())
user_max_limit = int(input())
user_m, user_n, user_min_limit, user_max_limit


def matrix_generator(
        m: int = 50,
        n: int = 50,
        min_limit: int = -250,
        max_limit: int = 1019) -> list[list[int]]:
    return [
        [randint(min_limit, max_limit) for _ in range(n)]
        for _ in range(m)
    ]
    
    
matrix_generator(
    user_m, user_n,
    user_min_limit, user_max_limit
)

1
1
1
1


[[1]]

### Задание №3
Реализовать методы сортировки строк числовой матрицы в соответствии с
заданием. Оценить время работы каждого алгоритма сортировки и сравнить его со
временем стандартной функции сортировки. Испытания проводить на сгенерированных
матрицах.

In [28]:
import copy
from typing import Union, Any, NamedTuple

In [29]:
# Сортировка выбором.
# Берётся срез массива, в котором минимальный элемент переносят в самый левый угол,
# после чего срез уменьшается и цикл повторяется.
def selection_sort(a: list[int]) -> list[int]:
    arr_len: int = len(a)

    for i, elem in enumerate(a):
        minimum = min(range(i, arr_len), key=a.__getitem__)
        a[i], a[minimum] = a[minimum], elem

    return a

In [30]:
# Сортировка вставкой.
def insertion_sort(a: list[int]) -> list[int]:
    for i in range(1, len(a)):
        value = a[i]
        j = i - 1
        while j >= 0 and a[j] > value:
            a[j+1] = a[j]
            j -= 1
        a[j+1] = value

    return a

In [31]:
# Сортировка обменом. || Сортировка пузырьком.
def bubble_sort(a: list[int]) -> list[int]:
    changed = True

    while changed:
        changed = False

        for i in range(len(a) - 1):
            if a[i] > a[i+1]:
                a[i], a[i+1] = a[i+1], a[i]
                changed = True

    return a

In [32]:
# Сортировка Шелла.
def shell_sort(a: list[int]) -> list[int]:
    mid: int = len(a) // 2

    while mid:
        for i, elem in enumerate(a):
            while i >= mid and a[i - mid] > elem:
                a[i] = a[i - mid]
                i -= mid
            a[i] = elem

        if mid == 2:
            mid = 1
        else:
            mid //= 2

    return a


In [33]:
# Быстрая сортировка.
def quick_sort(a: list[int], p: int=0, r: Union[int, None]=None):
    if r == None:
        r = len(a) - 1

    if p < r:
        q = partition(a, p, r)
        quick_sort(a, p, q-1)
        quick_sort(a, q+1, r)

    return a

    
def partition(a: list[int], p: int, r: int) -> int:
    pivot: int = a[r]
    i = p

    for j in range(p, r):
        if a[j] <= pivot:
            a[i], a[j] = a[j], a[i]
            i += 1

    a[i], a[r] = a[r], a[i]

    return i

In [34]:
class PriorityHeap:
    def __init__(self) -> None:
        self.cache: list[int] = []
        self.size: int = 0
        self.max_size = 32

    def pop_min(self) -> Any:
        res = self.cache[0]
        self.size -= 1
        self.cache[0] = self.cache[self.size]
        self.cache.pop(self.size)
        self.iter_heapify(0)
        return res

    def max_heap_insert(self, key: int):
        self.cache.append(-10000000000000)
        self.insert_key(key, self.size)
        self.size += 1


    def insert_key(self, value: int, i: int):
        self.cache[i] = value
        parent = self.parent(i)
        while i > 0 and self.cache[parent] > self.cache[i]:
            self.cache[parent], self.cache[i] = self.cache[i], self.cache[parent]
            i = parent
            parent = self.parent(i)

    def parent(self, i: int) -> int:
        res = (i >> 1) - 1
        if res < 0:
            return 0
        else:
            return res

    def iter_heapify(self, i: int) -> None:
        while True:
            l: int = (i << 1) + 1
            r: int = (i << 1) + 2
            largest: int = i

            if l < self.size and self.cache[l] < self.cache[largest]:
                largest = l

            if r < self.size and self.cache[r] < self.cache[largest]:
                largest = r

            if largest == i:
                break
            else:
                self.cache[i], self.cache[largest] = self.cache[largest], self.cache[i]
                i = largest

In [35]:
class TournamentBracket(NamedTuple):
    winners: list[int]
    losers: list[int]
    heap: PriorityHeap
    players_count: int
    players: list[int]

In [45]:
def fill_up_heap(b: TournamentBracket) -> TournamentBracket:
    while b.heap.size < b.heap.max_size and b.heap.size < b.players_count:
        elem = b.players[b.heap.size]
        if b.winners and b.winners[-1] > elem:
            b.losers.append(elem)
        else:
            b.heap.max_heap_insert(b.players[b.heap.size])

    return b


def add_more_players(b: TournamentBracket) -> TournamentBracket:
    for i in range(b.heap.size, b.players_count):
        elem: int = b.players[i]

        if b.winners and b.winners[-1] > elem:
            b.losers.append(elem)
        else:
            max_elem: int = b.heap.pop_min()

            if not b.winners:
                b.winners.append(max_elem)
            elif b.winners[-1] <= max_elem:
                b.winners.append(max_elem)
            else:
                b.losers.append(max_elem)

            b.heap.max_heap_insert(elem)

    return b


def get_last_players(b: TournamentBracket) -> TournamentBracket:
    while b.heap.size > 0:
        max_elem: int = b.heap.pop_min()

        if not b.winners:
            b.winners.append(max_elem)
        elif b.winners[-1] <= max_elem:
            b.winners.append(max_elem)
        else:
            b.losers.append(max_elem)

    return b


def find_winners_in_losers(b: TournamentBracket) -> list[int]:
    winners2: list[int] = tournament_sort(b.losers)

    i: int = 0
    j: int = 0
    winners1_len: int = len(b.winners)
    winners2_len: int = len(winners2)

    winners_res: list[int] = []

    while i < winners1_len and j < winners2_len:
        if b.winners[i] < winners2[j]:
            winners_res.append(b.winners[i])
            i += 1
        else:
            winners_res.append(winners2[j])
            j += 1

    while i < winners1_len:
        winners_res.append(b.winners[i])
        i += 1

    while j < winners2_len:
        winners_res.append(winners2[j])
        j += 1

    return winners_res

In [41]:
def tournament_sort(a: list[int]) -> list[int]:
    bracket: TournamentBracket = TournamentBracket(
        [],
        [],
        PriorityHeap(),
        len(a),
        a
    )

    bracket = fill_up_heap(bracket)
    bracket = add_more_players(bracket)
    bracket = get_last_players(bracket)

    if not bracket.losers:
        return bracket.winners
    else:
        return find_winners_in_losers(bracket)

In [57]:
def python_sort(a: list[int]) -> list[int]:
    a.sort()
    return a


def main() -> None:
    sorts_list: list[Callable] = list([
        selection_sort,
        insertion_sort,
        bubble_sort,
        shell_sort,
        tournament_sort,
        quick_sort,
    ])

    for sort_func in sorts_list:
        print(f"Testing sort: {sort_func.__name__}")
        custom_times: list[float] = []
        python_times: list[float] = []

        for row in matrix_generator():
            a: list[int] = list(x for x in row)
            a_copy: list[int] = a.copy()

            custom_start_time: float = time.time()
            undertest = sort_func(a_copy)
            custom_end_time: float = (time.time() - custom_start_time)

            python_start_time: float = time.time()
            expected = python_sort(a)
            python_end_time: float = (time.time() - python_start_time)

            assert expected == undertest, f"Sort {sort_func.__name__} is wrong"
            custom_times.append(custom_end_time)
            python_times.append(python_end_time)

        print(f"--- custom sort: {sum(custom_times) / 50 * 1000} ms ---")
        print(f"--- python sort: {sum(python_times) / 50 * 1000} ms ---")

In [58]:
main()

Testing sort: selection_sort
--- custom sort: 0.0917673110961914 ms ---
--- python sort: 0.003509521484375 ms ---
Testing sort: insertion_sort
--- custom sort: 0.12763500213623047 ms ---
--- python sort: 0.004734992980957031 ms ---
Testing sort: bubble_sort
--- custom sort: 0.3096914291381836 ms ---
--- python sort: 0.0038337707519531246 ms ---
Testing sort: shell_sort
--- custom sort: 0.07265090942382812 ms ---
--- python sort: 0.0033903121948242188 ms ---
Testing sort: tournament_sort
--- custom sort: 0.3055715560913086 ms ---
--- python sort: 0.003662109375 ms ---
Testing sort: quick_sort
--- custom sort: 0.05950450897216797 ms ---
--- python sort: 0.0031280517578125 ms ---


### Вывод


## Среднее время выполнения


| Сортировка | Время |
-------- | ----- |
Стандартная функция | 0.003509521484375 ms |
Выбором | 0.003509521484375 ms |
Вставкой | 0.12763500213623047 ms |
Обменом | 50.3096914291381836 ms |
Шелла | 0.07265090942382812 ms |
Турнирная | 0.3055715560913086 ms |
Быстрая | 0.05950450897216797 ms |


Результаты получены при сортировке списка длиной 50 элементов в диапазоне [-250:1019]

Таким образом, сортировки вставкой, обменом и турнирная получились наихудшими, наилучший результат показала сортировка выбором, которая одна из лучших для коротких списков, на следующем месте после неё стоит быстрая сортировка